# Comparing Multi-Agent System to a Single-Model Baseline for Code Generation

This notebook presents a comparative study evaluating the performance of a multi-agent system for automated code generation against a single large language model (LLM) baseline. The multi-agent system, designed with specialized agents for tasks such as preprocessing, retrieval, planning, coding, and debugging, aims to tackle complex coding challenges more effectively than a monolithic single-model approach.

The objective of this study is to quantify the difference in performance between the two approaches across a set of coding problems. We will evaluate performance based on metrics such as the success rate in generating correct code, the time taken to produce a solution, and potentially other relevant factors like the number of iterations or debugging steps.

This notebook will cover:

1.  **Setup:** Loading necessary libraries, configuring the environment, and preparing the coding problems for evaluation.
2.  **Baseline (Single Model):** Implementing and running the single-model approach on the chosen coding problems.
3.  **Multi-Agent System:** Integrating and running the existing multi-agent system within the notebook environment.
4.  **Evaluation:** Running both approaches on the same set of problems and collecting performance data.
5.  **Analysis:** Comparing the collected data to draw conclusions about the effectiveness of each approach.
6.  **Results and Discussion:** Presenting the findings and discussing the implications of the results.


## Setup

In [1]:
import os
import sys
import dotenv
import json
from loguru import logger

# Append the models path in order to import the models
PROJECT_ROOT = os.path.join(os.getcwd(), 'src/')
print(PROJECT_ROOT)
sys.path.append(PROJECT_ROOT)

# Load env variables
dotenv.load_dotenv()

# Set log level INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

/home/twanh/workspace/thesis/thesis-advent-of-agents/src/


1

In [2]:
# Import models from the system
from models.base_model import BaseLanguageModel
from models.gemini_model import GeminiLanguageModel
from models.openai_model import OpenAILanguageModel
from models.deepseek_model import DeepseekLanguageModel

### Load puzzles and input/outputs

In [3]:
# Get the correct paths
test_data_folder = os.path.join(PROJECT_ROOT, '..', 'experiments', 'test_data')
puzzles_folder = os.path.join(test_data_folder, 'puzzles/')
input_output_file = os.path.join(test_data_folder, 'answers2024.json')
puzzle_files = [os.path.join(puzzles_folder, f) for f in os.listdir(puzzles_folder) if os.path.isfile(os.path.join(puzzles_folder, f))]

In [4]:
# Create a datastructure were we can get by day
json_data = {}
with open(input_output_file, 'r') as f:
    json_data = {item['day']: item for item in json.load(f)}

puzzle_data = []
for file_path in puzzle_files:
    # Get the day of the puzzle file
    file_name = os.path.basename(file_path)
    day_str = file_name.split('_')[-1].split('.')[0]
    day = int(day_str)

    if day in json_data:
        with open(file_path, 'r') as f:
            puzzle_description = f.read()

        puzzle_info = {
            "year": json_data[day]['year'],
            "day": day,
            "description": puzzle_description,
            "input": json_data[day]['input'],
            "expected_output": json_data[day]['part1']
        }

        puzzle_data.append(puzzle_info)

# Sort by day
puzzle_data.sort(key=lambda x: x['day'])
print(len(puzzle_data)) # should be 25

25


### Model Configurations

Create the configurations for the baseline and the system to use. Since the advent of agents system can use multiple models for each agent, the model used for each agent will be the same as the single model used for the baseline.

The models that will be tested are:

<!-- TODO: Update model list -->
- Gemini
- OpenAI

In [5]:
from main import _get_model

models_to_test = ('gemini-2.0-flash', 'gemini-2.5-flash-preview-04-17', 'gemini-2.5-pro-preview-05-06' ,'gpt-4.1-mini')

configs = []
for model in models_to_test:
    configs.append({
        'baseline': _get_model(model),
        'system': {
            'preprocess': _get_model(model),
            'retreival': _get_model(model),
            'planning': _get_model(model),
            'coding': _get_model(model),
            'debugging': _get_model(model),
        }
    }) 
print(configs)
print(len(configs))

[{'baseline': <models.gemini_model.GeminiLanguageModel object at 0x701702110590>, 'system': {'preprocess': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f46c30>, 'retreival': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f476e0>, 'planning': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f781d0>, 'coding': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f78c80>, 'debugging': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f79760>}}, {'baseline': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f7a210>, 'system': {'preprocess': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f7acc0>, 'retreival': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f7b770>, 'planning': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f88260>, 'coding': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f88ce0>, 'debugging': <models.gemini_model.GeminiLanguageModel object at 0x7016e3f89730>}}, {'basel

## Baseline

### Baseline prompt

The prompt is based on the prompt the advent of agents system uses. However the baseline has no acess to the information the other agents provide. So there is only the `full_description` that provides the model with the puzzle. 
The steps in the prompt are the same expect the first step is removed, which was to analyze the generated plan.

In [6]:
BASELINE_PROMPT = """
# Advent of Code Implementation Agent

You are an expert coding agent specializing in implementing solutions for Advent of Code puzzles.
Your task is to convert a detailed solution plan into clean, efficient, and correct Python code that solves the given problem.
You excel at translating algorithmic plans into precise implementations.


It will be provided as the following JSON

```json
{{
    "full_description": "The full description of the problem (string)",
}}
```

## YOUR RESPONSIBILITIES

Your primary goal is to produce a complete, correct, and efficient Python implementation that:

1. Correctly solves both the provided examples and will work for the actual puzzle input
2. Follows good software engineering practices
3. Includes appropriate comments and documentation
4. Handles edge cases and potential errors
5. Is executable via command line as: `python3 [program].py [puzzleinputfile]`

## IMPLEMENTATION PROCESS

Follow these steps meticulously:

-----------------------------------------
STEP 1. Design Your Code Structure
-----------------------------------------

- Create a clear, modular structure with well-named functions matching the plan's major steps
- Define appropriate data structures with explicit type hints
- Plan your function signatures and interfaces before implementation
- Use the keywords and underlying concepts to think about what algoritms to use to solve the problems.

-----------------------------------------
STEP 2. Implement Core Logic
-----------------------------------------

- Write robust implementations of all algorithms described in the plan
- Include detailed comments explaining complex logic
- Follow Python best practices (PEP 8, appropriate naming conventions)
- Use type hints throughout your code


-----------------------------------------
STEP 3. Handle Edge Cases Explicitly
-----------------------------------------

- Add specific code to handle all edge cases mentioned in the plan
- Anticipate and handle additional edge cases common in Advent of Code:
  - Empty input
  - Boundary conditions (min/max values)
- Use the test cases to reason about your code and make sure it would solve the test cases correctly

----------------------------------------
STEP 4. Test Against Examples
----------------------------------------

- Include code that runs and validates against all provided examples
- Add assertions to verify intermediate results match expected values
- Print debugging information that would help diagnose issues to STDERR
    - STDOUT can only be used to print the final result.

----------------------------------------
STEP 5. Optimize If Necessary
-----------------------------------------

- Review your solution for performance bottlenecks
- Apply optimizations where appropriate, explaining your choices
- Ensure the solution will scale to handle the full problem input

-----------------------------------------
STEP 6. Finalize Solution
-----------------------------------------

- Ensure your code has a clear entry point (typically a `main()` function)
- Include code to read from the puzzle input file specified as a command-line argument
- Make sure that your code follows the proper structure as documented (example code template) below.
- Add a brief summary comment at the top explaining the approach
- Verify all functions have appropriate docstrings


-----------------------------------------
OUTPUT FORMAT
-----------------------------------------
Your response must be a valid JSON object with the following structure:

The generated code should be provided as the value of the code key in the JSON object. Ensure that the code is properly escaped to be a valid JSON string. This means that any double quotes within the code should be escaped with a backslash (\"), and newlines should be represented as \\n

```json
{{
  "code": "Complete Python code as a string with all necessary formatting. MAKE SURE THAT THIS IS VALID JSON"
}}
```


## EXAMPLE CODE TEMPLATE

```python
\"\"\"
Advent of Code [Year] Day [Number]: [Title]
Solution implementation based on the provided plan.

Usage: python3 solution.py [input_file]
\"\"\"
from typing import List, Dict, Tuple, Set, Optional
import sys
from collections import defaultdict, deque
import re
# Import other necessary libraries

def parse_input(input_file: str) -> [appropriate_return_type]:
    "\"\"Parse the puzzle input from file into appropriate data structures.

    Args:
        input_file: Path to the input file

    Returns:
        [Description of return value]
    \"\"\"
    with open(input_file, 'r') as f:
        # Process file content
        pass
    # Implementation...

def solve_part_one(parsed_data: [type]) -> [type]:
    \"\"\"Solve part one of the puzzle.

    Args:
        parsed_data: Processed input data

    Returns:
        Solution for part one
    \"\"\"
    # Implementation...

def main():
    # Check command line arguments
    if len(sys.argv) < 2:
        print("Usage: python3 solution.py [input_file]")
        return

    input_file = sys.argv[1]

    # Parse input
    parsed_data = parse_input(input_file)

    # Solve part one
    part_one_solution = solve_part_one(parsed_data)
    # ONLY PRINT THE RESULT, NO OTHER TEXT
    print(part_one_solution)

    # Test with examples (if available)
    # [Example testing code]

if __name__ == "__main__":
    main()
```

Remember to follow the plan closely while filling in implementation details that the planner may have omitted. Your goal is to bridge the gap between algorithmic description and working code.


-----------------------------------

Your input is:

{json_input}

"""

### Running the baseline

In [7]:
from utils.utils import extract_json_from_markdown
from agents.debugging_agent import DebuggingAgent
from utils.util_types import TestCase

def run_and_test_baseline(puzzle: str, puzzle_input: str, expected_output: str, model: BaseLanguageModel) -> bool:

    # Create the prompts
    json_inp = json.dumps({'full_description': puzzle})
    prompt = BASELINE_PROMPT.format(json_input=json_inp)
    # Prompt the model
    resp = model.prompt(prompt)

    # Extract the solution
    try:
        code = json.loads(extract_json_from_markdown(resp)[0]).get('code')
    except json.JSONDecodeError:
        print("Failed because decode")
        # TODO: Add retrying?
        return False

    # Use debugging agent to test the final solution
    dba = DebuggingAgent(
        'debugging',
        model=model,
        expected_output=expected_output,
        puzzle_input=puzzle_input,
    )
    run_result = dba._run_test(
        code,
        TestCase(
            input_=puzzle_input,
            expected_output=expected_output,
        ),
    )

    return run_result.success


In [8]:
baseline_results = {}
total = 0
solved = 0
# TODO: remove slice when running full tests
for config in configs[:1]:
    baseline_model = config['baseline']
    baseline_results[baseline_model.model_name] = {}
    print(f"Using model: {baseline_model.model_name}")
    for puzzle in puzzle_data:
        total += 1
        print(f"Running day {puzzle['day']}")
        success = run_and_test_baseline(
            puzzle=puzzle['description'],
            puzzle_input=puzzle['input'],
            expected_output=str(puzzle['expected_output']),
            model=baseline_model
        )
        baseline_results[baseline_model.model_name][puzzle['day']] = success
        if success:
            print("Solved!")
            solved += 1
        else:
            print("Not solved")

print(f"Solved: {solved}/{total}")

Using model: gemini-2.0-flash
Running day 1


2025-05-19 11:28:45.169 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:28:45.172 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:28:45.244 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 1646452=1646452


Solved!
Running day 2


2025-05-19 11:28:49.242 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:28:49.245 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:28:49.364 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 524=524


Solved!
Running day 3
Failed because decode
Not solved
Running day 4


...

Not solved
Running day 24


2025-05-19 11:31:11.193 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:31:11.195 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:31:11.264 | INFO     | agents.debugging_agent:_run_test:345 - Test case is successful 54715147844840=54715147844840


Solved!
Running day 25


2025-05-19 11:31:17.338 | INFO     | agents.debugging_agent:_run_test:337 - Running code with test case
2025-05-19 11:31:17.342 | INFO     | agents.debugging_agent:_run_code:307 - Running code
2025-05-19 11:31:17.433 | WARNING  | agents.debugging_agent:_run_test:357 - Test case was not successful
2025-05-19 11:31:17.433 | INFO     | agents.debugging_agent:_run_test:358 - Got: 22250, expected: 2900


Not solved
Solved: 11/25
